In [1]:
from pyspark.sql import SparkSession as sc
from pyspark.sql.functions import round, sum, lag
from pyspark.sql.window import Window

spark = sc.builder.getOrCreate()

25/04/02 13:38:43 WARN Utils: Your hostname, Mac-mini-Aleksandr.local resolves to a loopback address: 127.0.0.1; using 192.168.1.125 instead (on interface en1)
25/04/02 13:38:43 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/04/02 13:38:43 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
df = spark.read.csv("./covid.csv", header=True)

# Задание 1. 
Выберите 15 стран с наибольшим процентом переболевших на 31 марта (в выходящем датасете необходимы колонки: iso_code, страна, процент переболевших)

In [4]:
df_res_1 = df.select(
    df.iso_code,
    df.location,
    # df.date,
    # df.population,
    # df.total_cases,
    round((df.total_cases / df.population * 100), 3).alias("percent"),
).where(df.date == "2021-03-31")

In [5]:
df_res_1.orderBy(df_res_1.percent.desc()).show(15)

+--------+-------------+-------+
|iso_code|     location|percent|
+--------+-------------+-------+
|     AND|      Andorra| 15.544|
|     MNE|   Montenegro| 14.524|
|     CZE|      Czechia| 14.309|
|     SMR|   San Marino| 13.937|
|     SVN|     Slovenia| 10.371|
|     LUX|   Luxembourg|  9.847|
|     ISR|       Israel|  9.625|
|     USA|United States|  9.203|
|     SRB|       Serbia|  8.826|
|     BHR|      Bahrain|  8.489|
|     PAN|       Panama|  8.229|
|     PRT|     Portugal|  8.059|
|     EST|      Estonia|  8.023|
|     SWE|       Sweden|   7.97|
|     LTU|    Lithuania|  7.939|
+--------+-------------+-------+
only showing top 15 rows



# Задание 2.  
Top 10 стран с максимальным зафиксированным кол-вом новых случаев за последнюю неделю марта 2021 в отсортированном порядке по убыванию
(в выходящем датасете необходимы колонки: число, страна, кол-во новых случаев)

In [6]:
df_res_2 = df.select(df.date, df.location, df.new_cases).where(
    (df.date <= "2021-03-31") & (df.date >= "2021-03-25") & (df.continent != "NULL")
)

In [7]:
window = Window.partitionBy(df_res_2.location).orderBy(df_res_2.date)

In [8]:
df_res_2 = df_res_2.withColumn("sum_new_cases", sum(df_res_2.new_cases).over(window))

In [9]:
df_res_2.select(df_res_2.date, df_res_2.location, df_res_2.sum_new_cases).where(
    df_res_2.date == "2021-03-31"
).orderBy(df_res_2.sum_new_cases.desc()).show()

+----------+-------------+-------------+
|      date|     location|sum_new_cases|
+----------+-------------+-------------+
|2021-03-31|       Brazil|     528736.0|
|2021-03-31|United States|     448300.0|
|2021-03-31|        India|     434131.0|
|2021-03-31|       France|     266069.0|
|2021-03-31|       Turkey|     225900.0|
|2021-03-31|       Poland|     201046.0|
|2021-03-31|        Italy|     144037.0|
|2021-03-31|      Germany|     120656.0|
|2021-03-31|      Ukraine|      95016.0|
|2021-03-31|    Argentina|      78944.0|
|2021-03-31|         Peru|      67548.0|
|2021-03-31|  Philippines|      62977.0|
|2021-03-31|         Iran|      62247.0|
|2021-03-31|       Russia|      60870.0|
|2021-03-31|      Hungary|      58723.0|
|2021-03-31|  Netherlands|      54142.0|
|2021-03-31|     Colombia|      53167.0|
|2021-03-31|        Spain|      50034.0|
|2021-03-31|       Jordan|      48720.0|
|2021-03-31|        Chile|      47755.0|
+----------+-------------+-------------+
only showing top

# Задание 3.
Посчитайте изменение случаев относительно предыдущего дня в России за последнюю неделю марта 2021. (например: в россии вчера было 9150 , сегодня 8763, итог: -387) (в выходящем датасете необходимы колонки: число, кол-во новых случаев вчера, кол-во новых случаев сегодня, дельта)

In [17]:
df_res_3 = df.select(df.date, df.location, df.new_cases.alias("new_cases_today")).where(
    (df.location == "Russia") & (df.date.between("2021-03-24", "2021-03-31"))
)

In [18]:
window = Window.partitionBy(df_res_3.location).orderBy(df_res_3.date)

In [19]:
df_res_3 = df_res_3.withColumn("prev_new_cases", lag("new_cases_today", 1).over(window))

df_res_3 = df_res_3.select(
    "*", (df_res_3.new_cases_today - df_res_3.prev_new_cases).alias("delta")
)

In [23]:
df_res_3.where(df.date >= '2021-03-25').show()

+----------+--------+---------------+--------------+------+
|      date|location|new_cases_today|prev_new_cases| delta|
+----------+--------+---------------+--------------+------+
|2021-03-25|  Russia|         9128.0|        8769.0| 359.0|
|2021-03-26|  Russia|         9073.0|        9128.0| -55.0|
|2021-03-27|  Russia|         8783.0|        9073.0|-290.0|
|2021-03-28|  Russia|         8979.0|        8783.0| 196.0|
|2021-03-29|  Russia|         8589.0|        8979.0|-390.0|
|2021-03-30|  Russia|         8162.0|        8589.0|-427.0|
|2021-03-31|  Russia|         8156.0|        8162.0|  -6.0|
+----------+--------+---------------+--------------+------+

